In [1]:
# Importing neccessary libraries
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation

Problem 1


<div align="center">
  
 TOTAL VARIATION DIMINISHING SCHEME

</div>

The idea of the total variation diminishing scheme is to use a scheme in which stability is achieved by reducing the oscillations in the solution. To measure oscillations, the quantity defined is as follows:

$$ \boxed{TV(q) = \sum_{i=1}^N |q_i - q_{i-1}|} $$


This formula calculates the total variation by summing the absolute differences between consecutive elements in the solution.If $q_1$ and $q_N$ are taken to be constant,then, as long as the function remains monotonic, the $TV (q)$ is constant. However, if the values of qi develop local minima or maxima, then the TV increases. Thus, a scheme is said to be total variation diminishing, if the following condition holds:

$$ TV(q^{n+1}) < TV(q^{n}) $$


<div align="center">
  
 SLOPE AND FLUX LIMITERS

</div>
When a sharp change occurs, the solution gradients become very steep, which makes solution unstable. These errors arise as numerical methods fail to keep the track of the sharp changes and results into inacurate or unstable solution which is not physical solution. Hence, in order to achieve stability and accuracy, it becomes very important to limit certain quantities which resullts into these overflows
In order to ensure stability in the solution, sometimes, sudden changes in quantities are to bi limited. Limiters do exactly this job. They "limit" the overflow of quantities to achieve stability. 
The idea of slope limiter is to limit certain value of slope obtained at the interface which is obtained by calcuaing slope through adjecent quantites in question and get stability where as in flux limiter, 
the flux itself is limited to a certain value to achive stability. The difference between these two methods is only of the approach while numerically, they serve the same purpose. These concepts are closesly related to  one another in practise but are a bit different in principle. Moreover, Slope limiters are applied to individual grids while flux limiters are applied to grid surface

In [2]:
# Solving advection equation using finite difference method with periodic boundary conditions
def Flux_lim(c,nt,style):
    %matplotlib tk
    
    # Defining parameters
    a, b = -10, 10
    nx = 200
    x = np.linspace(a, b, nx)           # (b-a)/n gives surprisingly better result as it is more close and not just dt
    dx = x[1] - x[0]                                               
    t = np.linspace(0, 25, nt)          # (b-a)/n gives surprisingly better result as it is more close and not just dt
    dt = t[1] - t[0]

    # Defining top-hat as initial function
    pulse_height = 1.0
    pulse_width = 5.0
    q0 = np.where((np.abs(x) <= pulse_width), pulse_height, 0)

    # Plotting initial condition
    fig, ax = plt.subplots()
    line = ax.plot(x, q0, label="Initial Condition")[0]
    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"$f(x)$")
    ax.set_title(f"Fluxlimiter for {style} scheme at speed = {c}")

    # Copying the initial conditions for the array size and later updating them based on the function
    q = q0.copy()

    # Defining function that does the plotting
    def Algorithm(frame):
        qn = q.copy()
    
        # Initiating algorithm.
        for i in range(nx):

            # Manual Stability to avoid NAN values.
            dm = ((qn[i % nx] - qn[(i-1) % nx]))
            dp = ((qn[(i + 1) % nx]) - qn[(i) % nx])

            # Condition for positive speed.
            if c > 0:
                theta = 1
                # Condition for r_minus
                if dm != 0:
                    rm = (qn[(i - 1) % nx] - qn[(i - 2) % nx]) / dm
                else:
                    rm = 0.0
                # Condition for r_plus
                if dp != 0:
                    rp = (qn[(i) % nx] - qn[(i - 1) % nx]) / dp
                else:
                    rp = 0.0
            else:
            # Condition for negative speed.
                theta = -1
                # Condition for r_minus.
                if dm != 0:
                    rm = (qn[(i + 1) % nx] - qn[i % nx]) / dm
                else:
                    rm = -0.0
                # Condition for r_plus.
                if dp != 0:
                    rp = (qn[(i + 1) % nx] - qn[i % nx]) / dp
                else:
                    rp = -0.0           
            
            # Algorithm for particular scheme and for phi_plus and phi_minus.

            # Donner cell    
            if style == "Donner-cell":                          
                phip = 0
                phim = 0

            # Lax Wanderoff                                                
            elif style == "LaxW":                                                                       
                phip = 1
                phim = 1

            # Beam 
            elif style == "Beam":
                phip = rp
                phim = rm

            # Fromm
            elif style == "Fromm":                                   
                phip = (1 + rp)/2
                phim = (1 + rm)/2

            # Minmod                         
            elif style == "Minmod":                                            
                phip =  min(1,abs(rp))
                phim =  min(1,abs(rm))

            # Superbee
            elif style == "Superbee":                                          
                phip =  max(0,min(1,2*rp),min(2,rp))
                phim =  max(0,min(1,2*rm),min(2,rm))

            # Monotonic cebtral difference
            elif style == "MC":                                                          
                phip =  max(0,min((1 + rp)/2 ,2,2*rp))
                phim =  max(0,min((1 + rm)/2 ,2,2*rm))

            # Van Leer
            elif style == "Van Leer":                                   
                phip =  (rp + abs(rp)) / (1 + abs(rp))
                phim =  (rm + abs(rm)) / (1 + abs(rm))

            else:
                raise ValueError(f"No {style} style found.")             

            # Storing parameters in variable to put in final expression
            fp = c/2 * ((1 + theta)*qn[i % nx] + (1 - theta)*qn[(i + 1) % nx]) + abs(c)/2 * (1-abs(c*dt/dx))*(qn[(i + 1) % nx] - qn[i % nx]) * phip
            fm = c/2 * ((1 + theta)*qn[(i - 1) % nx] + (1 - theta)*qn[(i) % nx]) + abs(c)/2 * (1-abs(c*dt/dx))*(qn[i % nx] - qn[(i - 1) % nx]) * phim            
            
            # Final expression
            q[i] = qn[i] + (dt/dx) * (fm-fp)

        # Update the plot with the new solution
        line.set_ydata(q)
        return line
    
    # Create animation
    anim = FuncAnimation(fig, Algorithm, frames=nt, interval=50)

    # Show the animation
    return anim

In [4]:
# Example (not able to show all animations at once using this approach)
Output = Flux_lim(c = 3,nt = 1000,style = "Superbee")
plt.show()

Problem 2

#### A. 1D isothermal equations in their conservative forms
1. **Conservation of mass:**
   $$\frac{\partial \rho}{\partial t} + \frac{\partial (\rho v)}{\partial x} = 0$$

2. **Conservation of momentum:**
   $$\frac{\partial (\rho v)}{\partial t} + \frac{\partial (\rho v^2)}{\partial x} = -\frac{\partial P}{\partial x}$$

3. **Conservation of energy:**
   $$\frac{\partial (\rho e_{total})}{\partial t} + \frac{\partial (\rho v e_{total})}{\partial x} = -\frac{\partial (Pv)}{\partial x}$$

where:
- $\rho$ is the density of the fluid,
- $v$ is the velocity of the fluid in the x-direction,
- $P$ is the pressure of the fluid, given by $P = \rho c_0^2$
- $c_0$ is the Isothermal Sound Speed
- $e_{total}$ is the total energy per unit volume, which includes both internal energy and kinetic energy and is given by $e_{total} = e + \frac{1}{2} v^2$,
- $e$ is the internal/thermal energy per unit volume, given as $e = (\gamma -1) \frac{P}{\rho}$

#### B. CFL Condition
Given CFL Factor = 0.5, signal speed = $c=1$ and $\Delta x = 1$ with grid points = 100 and duration 30sec<br>
$\therefore$ using CFL Condition we have,<br>
$c_0 \frac{\Delta t}{\Delta x} \leq 0.5$<br>
Substituing the values of $\Delta x$ and $c_0$ for $\Delta t$, we have,<br>
$\Delta t = 0.5$

In [8]:
# The follwinng code is not working as expected. Tried multiple things, multiple methods and multiple smaller codes just to work once but i am not able to figure out the problem.
def Isothermal_1D(nt, style):
    %matplotlib tk
    # Defining parameters
    a, b = -49, 51
    nx = 102
    x = np.linspace(a, b, nx)
    dx = x[1] - x[0]
    t = np.linspace(0, 30, nt)
    dt = t[1] - t[0] 

    # Initial function
    threshold = 0
    q10 = 2 * np.where(x >= threshold, 1, 0) + 1
    q20 = np.zeros_like(q10)
    v0 = np.zeros_like(x)

    # Initial Plot
    fig, ax = plt.subplots()
    line = ax.plot(x, q10, label="Density")[0]
    ax.set_xlabel(r"$x$")
    ax.set_ylabel("Amplitude")
    ax.set_title(f"1D Isothermal Shock Tube using {style} scheme")

    # Set y-axis limits
    ax.set_ylim(0, 4)

    # Copying the initial conditions for the array size and later updating them based on the function
    q1 = q10.copy()
    q2 = q20.copy()

    # Defining algorithm that does the plotting
    def Algorithm(frames):
        # Copies of q1
        q1n = q1.copy()
        q1f = q1n.copy()

        # Copies of q2
        q2n = q2.copy()
        q2f = q2n.copy()

        # Initiating algorithm.
        for i in range(1, nx - 1):
            # Calculating velocity
            vp = 0.5 * (q2n[i + 1] / q1n[i + 1] + q2n[i] / q1n[i]) if (q1n[i + 1] != 0) and (q1n[i] != 0) else 0
            vm = 0.5 * (q2n[i] / q1n[i] + q2n[i - 1] / q1n[i - 1]) if (q1n[i] != 0) and (q1n[i - 1] != 0) else 0

            # Check for zero velocities and keeping alomst non zero
            if vp == 0:
                vp = 1e-6  
            if vm == 0:
                vm = 1e-6  

            # Manual Stability to avoid NAN values.
            dm1 = q1n[i] - q1n[i - 1]
            dp1 = q1n[i + 1] - q1n[i]

            dm2 = q2n[i] - q2n[i - 1]
            dp2 = q2n[i + 1] - q2n[i]

            # For vp
            # Condition for positive speed.
            if vp > 0:
                theta = 1
                # Condition for r_minus
                rm1 = (q1n[i - 1] - q1n[i - 2]) / dm1 if dm1 != 0 else 0.0
                # Condition for r_plus
                rp1 = (q1n[i] - q1n[i - 1]) / dp1 if dp1 != 0 else 0.0
            else:
                # Condition for negative speed.
                theta = -1
                # Condition for r_minus.
                rm1 = (q1n[i + 1] - q1n[i]) / dm1 if dm1 != 0 else -0.0
                # Condition for r_plus.
                rp1 = (q1n[i + 1] - q1n[i]) / dp1 if dp1 != 0 else -0.0
        
            # Condition for positive speed
            if vp > 0:
                theta = 1
                # Condition for r_minus
                rm2 = (q2n[i - 1] - q2n[i - 2]) / dm2 if dm2 != 0 else 0.0
                # Condition for r_plus
                rp2 = (q2n[i] - q2n[i - 1]) / dp2 if dp2 != 0 else 0.0
            else:
                # Condition for negative speed.
                theta = -1
                # Condition for r_minus.
                rm2 = (q2n[i + 1] - q2n[i]) / dm2 if dm2 != 0 else -0.0
                # Condition for r_plus.
                rp2 = (q2n[i + 1] - q2n[i]) / dp2 if dp2 != 0 else -0.0

            # For vm
            # Condition for positive speed.
            if vm > 0:
                theta = 1
                # Condition for r_minus
                rm1 = (q1n[i - 1] - q1n[i - 2]) / dm1 if dm1 != 0 else 0.0
                # Condition for r_plus
                rp1 = (q1n[i] - q1n[i - 1]) / dp1 if dp1 != 0 else 0.0
            else:
                # Condition for negative speed.
                theta = -1
                # Condition for r_minus.
                rm1 = (q1n[i + 1] - q1n[i]) / dm1 if dm1 != 0 else -0.0
                # Condition for r_plus.
                rp1 = (q1n[i + 1] - q1n[i]) / dp1 if dp1 != 0 else -0.0

            # For q2
            if vm > 0:
                theta = 1
                # Condition for r_minus
                rm2 = (q2n[i - 1] - q2n[i - 2]) / dm2 if dm2 != 0 else 0.0
                # Condition for r_plus
                rp2 = (q2n[i] - q2n[i - 1]) / dp2 if dp2 != 0 else 0.0
            else:
                # Condition for negative speed.
                theta = -1
                # Condition for r_minus.
                rm2 = (q2n[i + 1] - q2n[i]) / dm2 if dm2 != 0 else -0.0
                # Condition for r_plus.
                rp2 = (q2n[i + 1] - q2n[i]) / dp2 if dp2 != 0 else -0.0

            # Algorithm for particular scheme and for phi_plus and phi_minus.

            # Donner cell
            if style == "Donner-cell":
                phip1 = 0
                phim1 = 0
                phip2 = 0
                phim2 = 0

            # Lax Wanderoff
            elif style == "LaxW":
                phip1 = 1
                phim1 = 1
                phip2 = 1
                phim2 = 1

            # Beam
            elif style == "Beam":
                phip1 = rp1
                phim1 = rm1
                phip2 = rp1
                phim2 = rm1

            # Fromm
            elif style == "Fromm":
                phip1 = (1 + rp1) / 2
                phim1 = (1 + rm1) / 2
                phip2 = (1 + rp2) / 2
                phim2 = (1 + rm2) / 2

            # Minmod
            elif style == "Minmod":
                phip1 = min(1, abs(rp1))
                phim1 = min(1, abs(rm1))
                phip2 = min(1, abs(rp2))
                phim2 = min(1, abs(rm2))

            # Superbee
            elif style == "Superbee":
                phip1 = max(0, min(1, 2 * rp1), min(2, rp1))
                phim1 = max(0, min(1, 2 * rm1), min(2, rm1))
                phip2 = max(0, min(1, 2 * rp2), min(2, rp2))
                phim2 = max(0, min(1, 2 * rm2), min(2, rm2))

            # Monotonic cebtral difference
            elif style == "MC":
                phip1 = max(0, min((1 + rp1) / 2, 2, 2 * rp1))
                phim1 = max(0, min((1 + rm1) / 2, 2, 2 * rm1))
                phip2 = max(0, min((1 + rp2) / 2, 2, 2 * rp2))
                phim2 = max(0, min((1 + rm2) / 2, 2, 2 * rm2))

            # Van Leer
            elif style == "Van Leer":
                phip1 = (rp1 + abs(rp1)) / (1 + abs(rp1))
                phim1 = (rm1 + abs(rm1)) / (1 + abs(rm1))
                phip2 = (rp2 + abs(rp2)) / (1 + abs(rp2))
                phim2 = (rm2 + abs(rm2)) / (1 + abs(rm2))

            else:
                raise ValueError(f"No {style} style found.")

            # Storing parameters in variable to put in final expression for half step

            # For q1
            fp1 = vp / 2 * ((1 + theta) * q1n[i] + (1 - theta) * q1n[i + 1]) + abs(vp) / 2 * (1 - abs(vp * dt / dx)) * (q1n[i + 1] - q1n[i]) * phip1
            fm1 = vm / 2 * ((1 + theta) * q1n[i - 1] + (1 - theta) * q1n[i]) + abs(vm) / 2 * (1 - abs(vm * dt / dx)) * (q1n[i] - q1n[i - 1]) * phim1

            # For q2
            fp2 = vp / 2 * ((1 + theta) * q2n[i] + (1 - theta) * q2n[i + 1]) + abs(vp) / 2 * (1 - abs(vp * dt / dx)) * (q2n[i + 1] - q2n[i]) * phip2
            fm2 = vm / 2 * ((1 + theta) * q2n[i - 1] + (1 - theta) * q2n[i]) + abs(vm)/ 2 * (1 - abs(vm * dt / dx)) * (q2n[i] - q2n[i - 1]) * phim2

            # Final expression for half step
            xph = 0.5*(x[i + 1] + x[i])
            xmh = 0.5*(x[i] + x[i - 1])
            q1[i] = q1n[i] + (dt / (xph-xmh)) * (fm1 - fp1) # Tried dx as well, same result.
            q2[i] = q2n[i] + (dt / (xph-xmh)) * (fm2 - fp2) # Tried dx as well, same result.

            # Boundary conditions for half step
            q1[nx-1] = q1[nx-2]
            q1[0] = q1[1]

            q2f[nx-1] = q2f[nx-2]
            q2f[0] = q2f[1]
        

        # Full step expression with updated source term
        for j in range(1, nx - 1):
            xp = 0.5*(x[j + 1] + x[j])
            xm = 0.5*(x[j] + x[j - 1])
            q1f[j] = q1[j]
            q2f[j] = q1[j] - 0.5*((q1[j + 1] - q1[j - 1]) / (xp-xm)) # Tried dx as well, same result.]
            
            # Boundary condition for full step
            q1f[nx-1] = q1f[nx-2]
            q1f[0] = q1f[1]

            q2f[nx-1] = q2f[nx-2]
            q2f[0] = q2f[1]

        # Updating the plot with new solution
        line.set_ydata(q1f)
        return line

    # Creating animation
    anim = FuncAnimation(fig, Algorithm, frames=nt, interval=50)

    # Showing animation
    return anim


In [9]:
# Example
output = Isothermal_1D(60,"Superbee")
plt.show()

In [15]:
# Sirshaw's Code
def hydrosolver_animation(nt, dx, CFL, master_condition, show_html_video=False):
    %matplotlib tk
    def CapTheta(n):
        if n > 0:
            k = 1
        else:
            k = 0
        return k

    def phi_superbee(r):
        return np.where((r >= 0) & (r <= 1),
                    np.maximum(0, np.minimum(1, 2 * r)),
                    np.minimum(2, r))


    def flux_difference(q, dx, dt, nx):
        u_minus_half = np.zeros(nx)
        f_minus_half = np.zeros((2, nx))
        f_plus_half = np.zeros((2, nx))
        r_minus_half = np.zeros((2, nx))
        theta_minus_half = np.zeros(nx)
        for i in range(1, nx):
            u_minus_half[i] = 0.5 * ((q[1, i] / q[0, i]) + (q[1, i - 1] / q[0, i - 1]))  # defining speed at left interface by taking average

        for i in range(nx - 1):
            if u_minus_half[i] > 0:
                theta_minus_half[i] = 1.0
            else:
                theta_minus_half[i] = -1.0

        for i in range(2, nx - 2):
            for j in range(2):
                if u_minus_half[i] > 0:
                    if q[j, i] - q[j, i - 1] == 0:
                        r_minus_half[j, i] = (q[j, i - 1] - q[j, i - 2]) / 1e-16  # minfloat # dividing this small nubmer when denominator is zero
                    else:
                        r_minus_half[j, i] = (q[j, i - 1] - q[j, i - 2]) / (q[j, i] - q[j, i - 1])
                else:
                    if q[j, i] - q[j, i - 1] == 0:
                        r_minus_half[j, i] = (q[j, i + 1] - q[j, i]) / 1e-16  # minfloat # dividing this small number when denominator is zero
                    else:
                        r_minus_half[j, i] = (q[j, i + 1] - q[j, i]) / (q[j, i] - q[j, i - 1])

                f_minus_half[j, i] = 0.5 * u_minus_half[i] * ((1 + theta_minus_half[i]) * q[j, i - 1] + (
                            1 - theta_minus_half[i]) * q[j, i]) + 0.5 * np.abs(u_minus_half[i]) * (
                                              1 - np.abs(u_minus_half[i] * dt / dx)) * phi_superbee(
                    r_minus_half[j, i]) * (q[j, i] - q[j, i - 1])
                f_plus_half[j, i] = 0.5 * u_minus_half[i + 1] * ((1 + theta_minus_half[i + 1]) * q[j, i] + (
                            1 - theta_minus_half[i + 1]) * q[j, i + 1]) + 0.5 * np.abs(u_minus_half[i + 1]) * (
                                               1 - np.abs(u_minus_half[i + 1] * dt / dx)) * phi_superbee(
                    r_minus_half[j, i + 1]) * (q[j, i + 1] - q[j, i])
        df = f_plus_half - f_minus_half
        return df

    def Gradp(p, dx):
        y = np.zeros_like(p)
        for i in range(2, len(p) - 2):
            y[i] = 0.5 * (p[i + 1] - p[i - 1]) / dx
        return y

    # Defining Initial condition
    u0 = 0  # Taking initial speed as zero
    c0 = 1  # Sound speed
    N = 100
    x = np.arange(0, N + (2 * dx), dx)  # Here taking two ghost cells, as per the requirement...
    Time = np.zeros(nt)
    nx = len(x)

    rho = np.zeros(nx)
    for i in range(0, nx):
        rho[i] = 2 * CapTheta(50 - x[i]) + 1

    rhou = rho * u0

    q0 = np.zeros((2, nx))
    q0[0, :] = rho
    q0[1, :] = rhou

    q = np.zeros((nt, 2, nx))
    q[0] = q0

    fig, axs = plt.subplots(3, 1, figsize=(10, 15))
    plt.suptitle("Variation of Physical quantities")

    def animate(frame):
        axs[0].cla()
        axs[1].cla()
        axs[2].cla()

        for ax in axs:
            ax.set_xlim(0, 100)
            ax.set_xlabel('x')
            ax.grid()

        Den = q[frame, 0, :]
        P = q[frame, 0, :] * c0 ** 2
        U = q[frame, 1, :] / q[frame, 0, :]

        axs[0].plot(x, Den, 'k--', label=f'Density at time, t = {Time[frame]:.2f}', linewidth=3)
        axs[0].set_ylabel('Density')
        axs[0].legend()

        axs[1].plot(x, U, 'b--', label=f'Speed at time, t = {Time[frame]:.2f}', linewidth=3)
        axs[1].axhline(y=0, color='k', linestyle='--')
        axs[1].set_ylabel('Speed')
        axs[1].legend()

        axs[2].plot(x, P, 'r--', label=f'Pressure at time, t = {Time[frame]:.2f}', linewidth=3)
        axs[2].set_ylabel('Pressure')
        axs[2].legend()

    for n in range(nt - 1):
        u1 = q[n, 1, :] / q[n, 0, :]
        dt = CFL * min(dx / (np.abs(u1) + c0))
        Time[n + 1] = Time[n] + dt

        df = flux_difference(q[n, :, :], dx, dt, nx)
        q[n + 1, :, :] = q[n, :, :] - 0.5 * df * dt / dx

        p = q[n + 1, 0, :] * c0 ** 2
        q[n + 1, 1, :] = q[n + 1, 1, :] - Gradp(p, dx)

    # Applying boundary conditions
    if master_condition == "outflow":
        q[:, :, -1] = q[:, :, -2]
        q[:, :, 0] = q[:, :, 1]
        q[:, :, -2] = q[:, :, -3]
        q[:, :, 1] = q[:, :, 2]
    elif master_condition == "reflective":
        q[:, :, -1] = q[:, :, -3]
        q[:, :, 0] = q[:, :, 2]
    else:
        raise ValueError(f"No method type {master_condition}")

    anim = FuncAnimation(fig, animate, frames=nt, interval=400)

    if show_html_video:
        return HTML(anim.to_html5_video())
    else:
        return anim


In [14]:
Output = hydrosolver_animation(nt=500, dx=1.0, CFL=0.5, master_condition="reflective")